In [70]:
!pip install feedparser
!pip install yfinance
!pip install nltk
!pip install torch
import torch
import feedparser
import yfinance as yf
import nltk
from datetime import datetime
from collections import namedtuple
import requests
!pip install transformers -q
from transformers import pipeline
from nltk.tokenize import word_tokenize
import pandas as pd
import seaborn as sns
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\anuta\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [4]:
# Use the feedparser library to parse the RSS feed into a list of titles. 
feed_url = 'https://feeds.a.dj.com/rss/RSSMarketsMain.xml'
feed = feedparser.parse(feed_url)

# Create a struct for the pair (Ticker, Date Published) to help with lookup
Ticker_Data = namedtuple("Ticker_Time", ["ticker", "time", "sentiment"])

In [5]:
# From a company name, get the corresponding ticker. Source: https://stackoverflow.com/a/38968465
def get_ticker(company_name):
    url = "https://query2.finance.yahoo.com/v1/finance/search"
    user_agent = "Mozilla/5.0 (Windows NT 10.0; Win64; x64)"
    params = {"q": company_name, "quotes_count": 1, "country": "United States"}

    res = requests.get(url=url, params=params, headers={'User-Agent': user_agent})
    data = res.json()
    
    if len(data['quotes']) == 0:
        return None

    company_code = data['quotes'][0]['symbol']
    return company_code

In [6]:
# Greedily extract the ticker from an entry title.
def extract_ticker_name(title):
    tokens = word_tokenize(title)
    for token in tokens:
        potential_company = get_ticker(token)
        if potential_company is not None:
            return potential_company
    return None

In [57]:
# Fetch webpage content for sentiment analysis
def fetch_webpage(url):
    try:
        response = requests.get(url)
        if response.status_code == 200:
            return response.content
        else:
            print(f"Failed to retrieve webpage. Status code: {response.status_code}")
            return None
    except requests.exceptions.RequestException as e:
        print(f"Error fetching webpage: {e}")
        return None

In [14]:
# Create a list of Ticker_Time objects to keep track of the tickers and the corresponding dates that the article was published on
sentiment_pipeline = pipeline('sentiment-analysis')

def create_ticker_list(feed):
    ticker_times = []
    for entry in feed.entries:
        tick_name = extract_ticker_name(entry.title)
        if tick_name is None:
            continue
        else:
            sentiment = sentiment_pipeline(entry.description)
            tick_time = Ticker_Data(tick_name, entry.published, sentiment)
            ticker_times.append(tick_time)
    return ticker_times

ticker_list = create_ticker_list(feed)

No model was supplied, defaulted to distilbert/distilbert-base-uncased-finetuned-sst-2-english and revision af0f99b (https://huggingface.co/distilbert/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.


In [72]:
# Calculate the change in stock price between when the article was published and the current time.
def calculate_change(ticker_list):
    date = datetime.now().strftime('%Y-%m-%d')
    
    data = {
        'Ticker': [],
        'Price Change': [],
        'Article Sentiment': [],
        'Sentiment Score': []
    }

    for entry in ticker_list:
        t = yf.Ticker(entry.ticker)
        prices = t.history()
        if (len(prices) > 0):
            price_change = round(prices["Close"].iloc[len(prices)-1] - prices["Close"].iloc[0], 4)
            #print(str(entry.ticker) + " | Price Change: " + str(price_change) + " | Article Sentiment: " + str(entry.sentiment))
            data['Ticker'].append(entry.ticker)
            data['Price Change'].append(price_change)
            data['Article Sentiment'].append(entry.sentiment[0]["label"])
            data["Sentiment Score"].append(entry.sentiment[0]["score"])
            
    return pd.DataFrame(data)
df = calculate_change(ticker_list)

SXB=F: Period '1mo' is invalid, must be one of ['1d', '5d']
^SPUACTUT: Period '1mo' is invalid, must be one of ['1d', '5d']
DE000SLA4SG4.SG: Period '1mo' is invalid, must be one of ['1d', '5d']
